In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'AG-s2-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [4]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (156s),  symKL_DB_eta: 340.342,  symKL_DB_z: 439.348,  loss: -1420.640,  ess: 2.945,  kl_eta_ex: 1022.028,  kl_eta_in: 210.836,  kl_z_ex: 323.180,  kl_z_in: 39.814
epoch: 1\250 (153s),  symKL_DB_eta: 245.131,  symKL_DB_z: 258.451,  loss: -1169.949,  ess: 3.302,  kl_eta_ex: 666.289,  kl_eta_in: 204.196,  kl_z_ex: 177.694,  kl_z_in: 29.940
epoch: 2\250 (196s),  symKL_DB_eta: 186.616,  symKL_DB_z: 154.373,  loss: -1019.643,  ess: 3.647,  kl_eta_ex: 456.164,  kl_eta_in: 211.615,  kl_z_ex: 96.919,  kl_z_in: 22.455
epoch: 3\250 (216s),  symKL_DB_eta: 154.315,  symKL_DB_z: 102.943,  loss: -938.577,  ess: 3.920,  kl_eta_ex: 349.335,  kl_eta_in: 233.461,  kl_z_ex: 58.730,  kl_z_in: 17.523
epoch: 4\250 (152s),  symKL_DB_eta: 133.620,  symKL_DB_z: 80.156,  loss: -893.580,  ess: 4.091,  kl_eta_ex: 296.271,  kl_eta_in: 263.888,  kl_z_ex: 42.298,  kl_z_in: 14.766
epoch: 5\250 (146s),  symKL_DB_eta: 120.640,  symKL_DB_z: 67.014,  loss: -864.618,  ess: 4.222,  kl_eta_ex: 267.874,  kl_eta_

KeyboardInterrupt: 

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)